In [3]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [4]:
%reload_ext kedro.ipython


[01/15/24 16:29:44] INFO     Resolved project path as:                                              ]8;id=989043;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=941165;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py#146\146]8;;\
                             C:\Users\MonéMentz\dev\sandbox\kedro-pricing-app.                                     
                             To set a different path, run '%reload_kedro <project_root>'                           

[01/15/24 16:29:46] WARNING  C:\conda_envs\pricing_app\lib\site-packages\lazy_loader\__init__.py:78 ]8;id=889615;file://C:\conda_envs\pricing_app\lib\warnings.py\warnings.py]8;;\:]8;id=602598;file://C:\conda_envs\pricing_app\lib\warnings.py#109\109]8;;\
                             : KedroDeprecationWarning: 'ParquetDataSet' has been renamed to                       
                             'ParquetDataset', and the alias will be removed in Kedro-Datasets                     
                             2.0.0                                                                                 
                               attr = getattr(submod, name)                                                        
                                                                                                                   

                    INFO     Kedro project kedro-pricing-app                                        ]8;id=888973;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=970019;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py#115\115]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=333596;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=935470;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py#116\116]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=98753;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=800065;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\ipython\__init__.py#122\122]8;;\

In [9]:
df=catalog.load("acc_lvl_data")

[01/15/24 16:32:16] INFO     Loading data from acc_lvl_data (CSVDataset)...                     ]8;id=462133;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=143466;file://C:\conda_envs\pricing_app\lib\site-packages\kedro\io\data_catalog.py#482\482]8;;\

In [6]:
def calculate_payments(df):
    # Initialize a list to store the amortization schedules
    amortization_schedules = []

    # Apply the calculations to each row in the DataFrame
    for index, row in df.iterrows():
        # Initialize a dictionary to store the calculated values
        calculations = {
            't': [],
            't+1': [],
            'interest_payment': [],
            'principal_payment': [],
            'monthly_payment': [],
            'remaining_balance': [],
            'date_stamp': [],
            'interest_rate': [],
            'loan_term': [],
            'portfolio': [],
            'risk_grade': [],
            'mob': [],
            'param1': [],
            'param2': []
        }

        # Convert annual rate to monthly
        monthly_rate = row.interest_rate / 12

        # Calculate monthly payment
        monthly_payment = row.exposure * (monthly_rate / \
                                          (1 - (1 + monthly_rate) ** -row.loan_term))

        # Initialize the remaining balance as the loan amount
        remaining_balance = row.exposure

        # Initialize the t and t+1 counter
        t = 0
        t_plus_1 = t + 1

        # While there is still a balance remaining, calculate the interest and 
        #principal for the current payment
        while remaining_balance > 0:
            # Calculate the interest for the current month
            interest_payment = remaining_balance * monthly_rate

            # Calculate the principal for the current month
            principal_payment = monthly_payment - interest_payment

            # If the principal payment for this month is greater than \
            # the remaining balance, adjust the principal payment and \
            # the monthly payment
            if remaining_balance - principal_payment < 0:
                principal_payment = remaining_balance
                monthly_payment = principal_payment + interest_payment

            # Append the calculated values to the dictionary
            for key in calculations.keys():
                if key in ['t', 't+1']:
                    calculations[key].append(eval(key))
                elif key in ['interest_payment', 'principal_payment', \
                             'monthly_payment', 'remaining_balance']:
                    calculations[key].append(eval(key.lower().replace(' ', '_')))
                else:
                    calculations[key].append(getattr(row, \
                                                     key.lower().replace(' ', '_')))

            # Update the remaining balance
            remaining_balance -= principal_payment

            # Increment the t and t+1 counters
            t += 1
            t_plus_1 += 1

        # Create a new DataFrame with the calculated values
        amortization_schedule = pd.DataFrame(calculations)

        # Add the unique_id as a new column to the DataFrame
        amortization_schedule['unique_id'] = row.unique_id

        # Add the amortization schedule to the list
        amortization_schedules.append(amortization_schedule)

    # Concatenate all the amortization schedules into a single DataFrame
    amortization_schedules = pd.concat(amortization_schedules, ignore_index=True)

    return amortization_schedules

In [11]:
print(calculate_payments(df))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(calculate_payments(df))                                                                │
│   2                                                                                              │
│                                                                                                  │
│ in calculate_payments:29                                                                         │
│                                                                                                  │
│   26 │   │   monthly_rate = row.interest_rate / 12                                               │
│   27 │   │                                                                                       │
│   28 │   │   # Calculate monthly payment                                                         │
│ ❱ 29 │   │   monthly_payment = row.exposure * (monthly_rate / \                                  │
│   30 │   │   │   │   │   │   │   │   │   │     (1 - (1 + monthly_rate) ** -row.loan_term))       │
│   31 │   │                                                                                       │
│   32 │   │   # Initialize the remaining balance as the loan amount                               │
│                                                                                                  │
│ C:\conda_envs\pricing_app\lib\site-packages\pandas\core\generic.py:5989 in __getattr__           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'Series' object has no attribute 'exposure'

In [12]:
df2=catalog.lead(pd_data)

def rename_acc_lvl_columns(df):
    df = df.rename(columns={'loan_id': 'unique_id',
                            'loan_amount': 'exposure',
                            'account_open_date': 'date_stamp'})
    return df

def rename_pd_columns(df):
    df = df.rename(columns={'mob': 't+1'})
    return df

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 df2=catalog.lead(pd_data)                                                                   │
│    2                                                                                             │
│    3 def rename_acc_lvl_columns(df):                                                             │
│    4 │   df = df.rename(columns={'loan_id': 'unique_id',                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DataCatalog' object has no attribute 'lead'

In [14]:
df2=catalog.load(pd_data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 df2=catalog.load(pd_data)                                                                    │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pd_data' is not defined

In [15]:
catalog.load(pd_data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 catalog.load(pd_data)                                                                        │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'pd_data' is not defined